In [1]:
# First import the libraries that we need to use
import os
import pandas as pd
import time
from datetime import datetime
import requests
import json
import warnings

from utils.coinbase import sleep, fetch_product_trades, fetch_product_candles, fetch_product_ticker, fetch_product_stats, get_currencies, get_products

from dotenv import load_dotenv
load_dotenv()

cb_pro = os.getenv("COINBASE_PRO_FQDN")
cb_pub = os.getenv("COINBASE_PUB_FQDN")
cb_sandbox = os.getenv("COINBASE_SANDBOX_FQDN")

print(cb_pro)
print(cb_pub) # used for most calls
print(cb_sandbox)


https://api-pro.coinbase.com
https://api.exchange.coinbase.com
https://api-public.sandbox.exchange.coinbase.com


In [2]:
fetch_product_trades('BTC-USD')

Created CSV file: data/coinbase_BTC-USD_trades_2022-05-15.csv.


,time,trade_id,price,size,side,date
0,2022-05-15T21:11:23.393457Z,337684520,30937.90000000,0.00168000,buy,2022-05-15 21:11:23.393457+00:00
1,2022-05-15T21:11:23.340281Z,337684519,30941.00000000,0.07448142,sell,2022-05-15 21:11:23.340281+00:00
2,2022-05-15T21:11:23.340281Z,337684518,30940.84000000,0.02454899,sell,2022-05-15 21:11:23.340281+00:00
3,2022-05-15T21:11:23.272272Z,337684517,30937.27000000,0.00030000,sell,2022-05-15 21:11:23.272272+00:00
4,2022-05-15T21:11:23.220234Z,337684516,30937.27000000,0.00010220,sell,2022-05-15 21:11:23.220234+00:00
...,...,...,...,...,...,...
995,2022-05-15T21:10:01.660024Z,337683525,30992.64000000,0.00016070,sell,2022-05-15 21:10:01.660024+00:00
996,2022-05-15T21:10:01.28409Z,337683524,30990.22000000,0.00013180,buy,2022-05-15 21:10:01.284090+00:00
997,2022-05-15T21:10:01.096941Z,337683523,30992.95000000,0.00587646,sell,2022-05-15 21:10:01.096941+00:00
998,2022-05-15T21:10:01.096941Z,337683522,30991.66000000,0.00100000,sell,2022-05-15 21:10:01.096941+00:00


In [3]:
## Get full list of Currencies
currencies_df = get_currencies()
currencies_df.head()

Created CSV file: data/coinbase_currencies.csv


,id,name,min_size,status,max_precision
0,GALA,Gala,0.00000001,online,0.00000001
1,BCH,Bitcoin Cash,0.00000001,online,0.00000001
2,LPT,Livepeer,0.01,online,0.001
3,GLM,Golem Network,0.00000001,online,0.00000001
4,MDT,Measurable Data Token,1,online,0.00000001


In [4]:
## Get full list of Products and information
products_online_df = get_products()
products_online_df.head()

Created CSV file: data/coinbase_products.csv


,id,base_currency,quote_currency
0,SNX-EUR,SNX,EUR
1,BICO-USD,BICO,USD
2,SNX-BTC,SNX,BTC
3,REQ-GBP,REQ,GBP
4,VGX-USD,VGX,USD


In [5]:
# Get products that are USD
usd_products_df = products_online_df.loc[(products_online_df['quote_currency'] == 'USD')].copy().drop(columns=["quote_currency"])
usd_products_df.head()

,id,base_currency
1,BICO-USD,BICO
4,VGX-USD,VGX
5,SUSHI-USD,SUSHI
8,UNFI-USD,UNFI
9,CTSI-USD,CTSI


In [6]:
# Fetch some data
top_stonks = usd_products_df.head(10).sort_values(by=['id'])

for coins in top_stonks['id']:
    fetch_product_trades(coins)
    fetch_product_candles(coins)
    sleep(2)

Created CSV file: data/coinbase_ALCX-USD_trades_2022-05-15.csv.
Created CSV file: data/coinbase_ALCX-USD_candles_2022-05-15.csv.
Created CSV file: data/coinbase_BICO-USD_trades_2022-05-15.csv.
Created CSV file: data/coinbase_BICO-USD_candles_2022-05-15.csv.
Created CSV file: data/coinbase_CRO-USD_trades_2022-05-15.csv.
Created CSV file: data/coinbase_CRO-USD_candles_2022-05-15.csv.
Created CSV file: data/coinbase_CTSI-USD_trades_2022-05-15.csv.
Created CSV file: data/coinbase_CTSI-USD_candles_2022-05-15.csv.
Created CSV file: data/coinbase_CVC-USD_trades_2022-05-15.csv.
Created CSV file: data/coinbase_CVC-USD_candles_2022-05-15.csv.
Created CSV file: data/coinbase_FET-USD_trades_2022-05-15.csv.
Created CSV file: data/coinbase_FET-USD_candles_2022-05-15.csv.
Created CSV file: data/coinbase_REN-USD_trades_2022-05-15.csv.
Created CSV file: data/coinbase_REN-USD_candles_2022-05-15.csv.
Created CSV file: data/coinbase_SUSHI-USD_trades_2022-05-15.csv.
Created CSV file: data/coinbase_SUSHI-US

In [33]:
# BETA: Fetch product `ticker` dictionarys and merge into df
# Uses function: fetch_product_ticker() which is a "no polling" endpoint
# This gets the 10 most recent trades on thier platform.

top_stonks_df = usd_products_df.sort_values(by=['base_currency']).reset_index()
top_stonks_df = top_stonks_df.drop(columns='index').head(10)

warnings.filterwarnings('ignore') #You know, if we have to do this, we should probably just not.
top_stonks_ticker_df = pd.DataFrame(dtype='float64')

for coins in top_stonks_df['id']:
    top_stonks_ticker_df['id'] = coins
    top_stonks_ticker_df = top_stonks_ticker_df.append(fetch_product_ticker(coins), ignore_index=True)
    
top_stonks_ticker_df["id"] = top_stonks_ticker_df["id"].astype("string")
top_stonks_ticker_df["ask"] = top_stonks_ticker_df["ask"].astype("float")
top_stonks_ticker_df["bid"] = top_stonks_ticker_df["bid"].astype("float")
top_stonks_ticker_df["price"] = top_stonks_ticker_df["price"].astype("float")
top_stonks_ticker_df["size"] = top_stonks_ticker_df["size"].astype("float")
top_stonks_ticker_df["time"] = top_stonks_ticker_df["time"].astype("datetime64")
top_stonks_ticker_df["trade_id"] = top_stonks_ticker_df["trade_id"].astype("float")
top_stonks_ticker_df["volume"] = top_stonks_ticker_df["volume"].astype("float")
    
top_stonks_ticker_df = top_stonks_ticker_df.sort_values(by=['time'], ascending=False).reset_index().drop(columns='index').dropna()

# top_stonks_ticker_df.dtypes
top_stonks_ticker_df

,id,ask,bid,price,size,time,trade_id,volume
0,ALICE-USD,0.581300,0.581000,0.581000,6.2700,2022-05-15 21:37:38.377113,71022296.0,7.823533e+07
1,ALICE-USD,0.471400,0.471000,0.471000,17.3000,2022-05-15 21:37:37.478861,53486506.0,2.130196e+07
2,ALICE-USD,0.966000,0.964000,0.964000,11.3300,2022-05-15 21:37:31.970081,4982589.0,6.753969e+05
4,ALICE-USD,0.141000,0.140500,0.140900,14.2000,2022-05-15 21:36:54.278408,168863.0,8.098729e+05
5,ALICE-USD,0.016544,0.016505,0.016508,2922.3000,2022-05-15 21:36:52.400466,16397369.0,1.459319e+08
6,ALICE-USD,88.400000,88.390000,88.400000,145.4280,2022-05-15 21:36:51.611575,17300179.0,1.007412e+05
7,ALICE-USD,0.075300,0.075000,0.075000,48.7000,2022-05-15 21:36:42.489723,554360.0,1.623207e+06
8,ALICE-USD,0.500000,0.498000,0.499000,30.0000,2022-05-15 21:36:42.017025,1876634.0,1.579888e+06
9,ALICE-USD,36.060000,35.970000,35.960000,1.0056,2022-05-15 21:35:36.147383,1374365.0,1.540916e+04


In [30]:
top_stonks_ticker_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   index     4 non-null      int64         
 1   id        4 non-null      string        
 2   ask       4 non-null      float64       
 3   bid       4 non-null      float64       
 4   price     4 non-null      float64       
 5   size      4 non-null      float64       
 6   time      4 non-null      datetime64[ns]
 7   trade_id  4 non-null      float64       
 8   volume    4 non-null      float64       
dtypes: datetime64[ns](1), float64(6), int64(1), string(1)
memory usage: 320.0 bytes
